## This project uses Convolutional Neural Network to classify 25000 images into two categories Dog or Cat.

### Importing the necessary libraries

In [ ]:
import tensorflow as tf
import numpy as np
import cv2
import matplotlib.pyplot as plt
import os
import random
from sklearn.model_selection import train_test_split
import datetime
from tqdm import tqdm

### Checking the work of Tensorflow GPU

In [ ]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())a

# Pre-Processing

In [ ]:
DIR = ("B:\D2\PetImages")
training_data = []
CATEGORY = ["Cat", "Dog"]
IMG_SIZE = 100

for category in CATEGORY:
    path = os.path.join(DIR, category)
    label = CATEGORY.index(category)
    for img in tqdm(os.listdir(path)):
        try:
            img_array = cv2.imread(os.path.join(path,img), 0)
            new_array = cv2.resize(img_array, (IMG_SIZE, IMG_SIZE))
            training_data.append([new_array, label])
        except Exception as e:
            pass

In [ ]:
np.random.shuffle(training_data)

In [ ]:
inputs = []
targets = []

In [ ]:
for input, target in training_data:
    inputs.append(input)
    targets.append(target)

In [ ]:
type(inputs)


In [ ]:
inputs = np.array(inputs).reshape(-1, IMG_SIZE, IMG_SIZE, 1)

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(inputs, targets, test_size=0.1, random_state=1)

In [ ]:
train_inputs, validation_inputs, train_outputs, validation_outputs = train_test_split(X_train,Y_train, test_size=0.1, random_state=42)

In [ ]:
np.savez('B:\D2\PetImages\cats_and_dogs_train.npz', inputs = train_inputs, targets = train_outputs)
np.savez('B:\D2\PetImages\cats_and_dogs_validation.npz', inputs = validation_inputs, targets = validation_outputs)
np.savez('B:\D2\PetImages\cats_and_dogs_test.npz', inputs =  X_test, targets = Y_test)

# MODEL

In [ ]:
import tensorflow as tf
import numpy as np
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

In [ ]:
train_data = np.load('B:\D2\PetImages\cats_and_dogs_train.npz')
validation_data = np.load('B:\D2\PetImages\cats_and_dogs_validation.npz')


In [ ]:
train_inputs, train_targets = train_data['inputs'].astype(np.float), train_data['targets'].astype(np.int)
validation_inputs, validation_targets = validation_data['inputs'].astype(np.float), validation_data['targets'].astype(np.int)

In [ ]:

train_inputs_fit = train_inputs / 255.
validation_inputs = validation_inputs / 255.

In [ ]:
inputs_size = 100*100
outputs_size = 1
hidden_layer_size = 512

model = tf.keras.Sequential([
                            tf.keras.layers.Conv2D(64, kernel_size = (3,3), activation = 'relu'),
                            tf.keras.layers.MaxPool2D(pool_size = (2,2)),

                            tf.keras.layers.Conv2D(128, kernel_size = (3,3), activation = 'relu'),
                            tf.keras.layers.MaxPool2D(pool_size = (2,2)),
    
    
                            tf.keras.layers.Conv2D(256, kernel_size = (3,3), activation = 'relu'),
                            tf.keras.layers.MaxPool2D(pool_size = (2,2)),
    
                            tf.keras.layers.Flatten(),
                            tf.keras.layers.Dense(hidden_layer_size, activation='relu'),
                            tf.keras.layers.Dense(outputs_size, activation='sigmoid')
                            ])

BATCH_SIZE = 50
MAX_EPOCHS = 20
EARLY_STOPPING = tf.keras.callbacks.EarlyStopping(patience = 3)
model.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

### TRAINING

In [ ]:
model.fit(train_inputs_fit, train_targets, validation_data = (validation_inputs, validation_targets), epochs = MAX_EPOCHS, batch_size = BATCH_SIZE, callbacks=[EARLY_STOPPING])

### TESTING

In [ ]:
test_data = np.load('B:\D2\PetImages\cats_and_dogs_test.npz')
test_inputs, test_targets = test_data['inputs'].astype(np.float), test_data['targets'].astype(np.int)

In [ ]:
predictions = model.predict(test_inputs_final)test_loss, test_accuracy = model.evaluate(test_inputs_final, test_targets)
predictions = model.predict(test_inputs_final)

### SAVING THE CNN MODEL

In [ ]:
model.save('B:\D2\PetImages\cats_dogs.h5')